In [13]:
import csv

import numpy as np
import pandas as pd
import tensorflow as tf
from   tensorflow.keras.preprocessing.text import Tokenizer
from   tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
tf.__version__

'2.0.0'

In [4]:
!ls ../../../data

bbc-text.csv     caroll-alice.txt


In [7]:
#!head -5 ../../../data/bbc-text.csv

In [3]:
DATA = '../../../data'

In [4]:
VOCAB_SIZE = 2000
EMBEDDING_DIM = 50
MAX_LEN = 40
TRUNC_TYPE = 'post'
PADDING_TYPE = 'post'
OOV_TOK = '<UNK>'
TRAINING_PORTION = 0.8

In [38]:
sentences = []
labels = []
stopwords = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an",
    "and", "any", "are", "as", "at", "be", "because", "been", "before", 
    "being", "below", "between", "both", "but", "by", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from",
    "further", "had", "has", "have", "having", "he", "he'd", "he'll", 
    "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
    "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", 
    "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", 
    "most", "my", "myself", "nor", "of", "on", "once", "only", "or", 
    "other", "ought", "our", "ours", "ourselves", "out", "over", "own", 
    "same", "she", "she'd", "she'll", "she's", "should", "so", "some", 
    "such", "than", "that", "that's", "the", "their", "theirs", "them", 
    "themselves", "then", "there", "there's", "these", "they", "they'd", 
    "they'll", "they're", "they've", "this", "those", "through", "to", 
    "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
    "we're", "we've", "were", "what", "what's", "when", "when's", "where",
    "where's", "which", "while", "who", "who's", "whom", "why", "why's", 
    "with", "would", "you", "you'd", "you'll", "you're", "you've", "your",
    "yours", "yourself", "yourselves"]
len(stopwords)

153

In [39]:
with open(f'{DATA}/bbc-text.csv', 'r') as f:
    i = 0
    for line in f:
        if i != 0:
            lab, sent = line.split(',')
            sentences.append(sent[:-1])
            labels.append(lab)
        i += 1

In [40]:
print(len(labels), len(sentences))

2225 2225


In [42]:
sentences[0][:100]

'tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital'